In [6]:
import ccxt
import pandas as pd
import datetime


def fetch_ohlcv(exchange, symbol, timeframe, since, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
    return ohlcv

def to_dataframe(ohlcv_data):
    df = pd.DataFrame(ohlcv_data, columns=["timestamp", "open", "high", "low", "close", "volume"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df

def main():
    exchange = ccxt.bitflyer()
    symbol = "BTC/JPY"
    
    # 1時間足データの取得
    timeframe_hour = "1h"
    limit_hour = 30000  # 1時間足のデータを1000件取得
    since_hour = exchange.parse8601("2020-01-01T00:00:00Z")  # 2020年1月1日からのデータを取得
    ohlcv_data_hour = fetch_ohlcv(exchange, symbol, timeframe_hour, since_hour, limit_hour)
    df_hour = to_dataframe(ohlcv_data_hour)
    print("1時間足データ:")
    print(df_hour)
    
    # 15分足データの取得
    timeframe_15min = "15m"
    limit_15min = 30000  # 15分足のデータを1000件取得
    since_15min = exchange.parse8601("2020-01-01T00:00:00Z")  # 2020年1月1日からのデータを取得
    ohlcv_data_15min = fetch_ohlcv(exchange, symbol, timeframe_15min, since_15min, limit_15min)
    df_15min = to_dataframe(ohlcv_data_15min)
    print("15分足データ:")
    print(df_15min)

if __name__ == "__main__":
    main()

1時間足データ:
            timestamp       open       high        low      close     volume
0 2023-04-23 05:00:00  3712326.0  3712375.0  3707648.0  3711225.0  25.599508
1 2023-04-23 06:00:00  3711225.0  3711830.0  3709708.0  3709795.0   0.861117
15分足データ:
            timestamp       open       high        low      close     volume
0 2023-04-23 05:30:00  3713155.0  3713155.0  3707965.0  3709721.0  24.521348
1 2023-04-23 05:45:00  3709721.0  3711225.0  3707648.0  3711225.0   4.674163
2 2023-04-23 06:00:00  3711225.0  3711830.0  3709924.0  3711006.0   0.735498


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import ccxt
import pandas as pd
import numpy as np
from datetime import datetime


class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, d_model, nhead, num_layers):
        super().__init__()
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.input_fc = nn.Linear(input_dim, d_model)
        self.output_fc = nn.Linear(d_model, output_dim)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.transformer.encoder(x)
        x = self.output_fc(x[:, -1])
        return x


def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    return epoch_loss

def prepare_data(df, input_cols, output_cols, window_size):
    X = []
    y = []
    print(len(df))
    for i in range(len(df) - window_size):
        print(f"Processing row {i}")  # デバッグステートメントを追加
        X.append(df[input_cols].iloc[i:i + window_size].values)
        y.append(df[output_cols].iloc[i + window_size].values)

    X = np.array(X)
    y = np.array(y)

    # Split data into train and test sets
    split_index = int(X.shape[0] * 0.8)
    X_train = X[:split_index]
    X_test = X[split_index:]
    y_train = y[:split_index]
    y_test = y[split_index:]
    
    print("X_train length:", len(X_train))
    print("y_train length:", len(y_train))

    return X_train, X_test, y_train, y_test

def main():
    # Get historical data
    exchange = ccxt.bitflyer()
    symbol = "BTC/JPY"

    df_hour = pd.DataFrame(exchange.fetch_ohlcv(symbol, "1h", limit=1000), columns=["timestamp", "open", "high", "low", "close", "volume"])
    df_hour["timestamp"] = pd.to_datetime(df_hour["timestamp"], unit="ms")
    
    df_15min = pd.DataFrame(exchange.fetch_ohlcv(symbol, "15m", limit=1000), columns=["timestamp", "open", "high", "low", "close", "volume"])
    df_15min["timestamp"] = pd.to_datetime(df_15min["timestamp"], unit="ms")

    # Prepare data
    input_cols = ["open", "high", "low", "close", "volume"]
    output_cols = ["buy_price", "sell_price", "return"]
    window_size = 50

    df_hour["buy_price"] = df_hour["close"] + np.random.uniform(-100, 100, len(df_hour))
    df_hour["sell_price"] = df_hour["close"] + np.random.uniform(-100, 100, len(df_hour))
    df_hour["return"] = (df_hour["sell_price"] - df_hour["buy_price"]) / df_hour["buy_price"]

    X_train, X_test, y_train, y_test = prepare_data(df_hour, input_cols, output_cols, window_size)

    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Create model
    input_dim = len(input_cols)
    output_dim = len(output_cols)
    d_model = 128
    nhead = 4
    num_layers = 2

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TransformerModel(input_dim, output_dim, d_model, nhead, num_layers).to(device)

    # Prepare validation data
    X_train, X_test, y_train, y_test = prepare_data(df_hour, input_cols, output_cols, window_size)


    val_dataset = TensorDataset(X_val, y_val)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Train and validate model
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 50

    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, train_dataloader, criterion, optimizer, device)
        val_loss = evaluate(model, val_dataloader, criterion, device)
        print(f"Epoch {epoch}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Predict on new data
    new_data = df_15min[input_cols].iloc[-window_size:].values
    prediction = predict(model, new_data, device)
    print("Prediction (buy_price, sell_price, return):", prediction)

if __name__ == "__main__":
    main()

2
X_train length: 0
y_train length: 0


ValueError: num_samples should be a positive integer value, but got num_samples=0